In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=75d143476be13f3e91f047f6350df50230648b7408509b20e5f3785049cbba6d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


-----------------------------------------

-----------------------------------------

<b>SparkContext</b>
* Khởi tạo 1 SparkContext: Đại diện cho 1 phiên Spark và là điểm khởi đầu cho tất cả các hoạt động phân tán trong Spark.Nó là điểm kết nối giữa ứng dụng của bạn và cụm Spark. 

* Khi khởi tạo SparkContext, cần chỉ định một URL như "local" cho cục bộ hoặc 1 URL chính thức khác để kết nối với các nút khác. 

* Sau đó có thể sử dụng SparkContext để quản lý các RDD thực hiện các transform hay action 

* Từ Spark 2.0 trở đi thì thường ngta sẽ sử dụng SparkSession

In [13]:
from pyspark import SparkConf, SparkContext 


In [8]:
config = SparkConf().setMaster("local").setAppName("My App")

In [10]:
sc = SparkContext(conf = config)

<b>RDD</b>: là một CTDL cơ bản trong Spark
* Một vài tính chất cơ bản của RDD sẽ được tìm hiểu sau như ( immutable, hỗ trợ cả structure hoặc non-structure,....)

Một vài cách khởi tạo RDD:
*  sc.parallelize(): từ một danh sách, tập hợp dữ liệu 
* sc.textFile(): từ một hoặc nhiều tệp văn bản 
* sc.newAPIHadoopFile(): từ TextInputFormat của Hadoop
..........

In [16]:
rdd = sc.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:287

In [15]:
# lines = sc.textFile("path/to/textFile.txt")

<b>RDD Operations</b>: RDD hỗ trợ 2 loại hoạt động là <b><i>transformation</i></b> và <b><i>action</i></b>
* Transformation: Biến đổi RDD thành 1 RDD khác 

 <i>Ví dụ:</i> 
    * <b>map()</b> 
    * <b>filter()</b>

* Action: Là các hoạt động trả về kết qura cho chương trình hoặc ghi vào bộ lưu trữ và bắt đầu tính toán. 

 <i>Ví dụ:</i>
    * <b>count()</b>
    * <b>first()</b>


In [19]:
double_rdd = rdd.map(lambda x: x**2)
double_rdd.collect()

[1, 4, 9, 16, 25]

In [21]:
filters_rdd = rdd.filter(lambda x: x%2 != 0)
filters_rdd.collect()

[1, 3, 5]

In [23]:
count_rdd = rdd.count()
count_rdd 

5

In [24]:
first_rdd = rdd.first()
first_rdd 

1

-----------------------------------------

<strong>Lazy Evaluation</strong>: là khi chúng ta gọi một phép transformation trên RDD( ví dụ: <b>map()</b> ) thì thao tác này không được thực hiện ngay lập tức cho đến khi kết quả thực sự được yêu cầu. 

Như ví dụ ở trên khi rdd.map() được sử dụng thì Spark chỉ ghi nhớ phép biến đổi và chỉ khi có 1 hành động(actions) như in ra màn hình <b>collect()</b> Spark sẽ thực hiện toàn bộ chuỗi các phép biến đổi đã lập trong kế hoạch

Điều này có lợi ích gì ????

 -> Tối ưu hiệu suất: Chỉ tính toán khi cần

-----------------------------------------

<b>Fuction in Spark</b> 





In [29]:
rdd.collect()

[1, 2, 3, 4, 5]

In [31]:
### Using lambda with short function
result = rdd.filter(lambda x: x%2 != 0)
result.collect()


[1, 3, 5]

In [33]:
### Using def 
def square(s):
  return s%2 !=0
result = rdd.filter(square)
result.collect()

[1, 3, 5]

In [39]:
### Có một lưu ý đó là khi sử dụng 1 def của 1 đối tượng 
class MyClass:
  def __init__(self,value):
    self.value = value 
  def add(self,x):
    return self.value + x 
obj = MyClass(10)
result = rdd.map(obj.add)
print(result.collect())
### hiểu đơn giản là chúng ta đ truyền cả đối tượng obj để các node làm việc 
### và nếu class phức tạp -> lãng phí tài nguyên không cần thiết  

[11, 12, 13, 14, 15]


In [40]:
### Cách giải quyết là chỉ truyền hàm vào thôi 
result = rdd.map(lambda x: obj.add(x))
result.collect()


[11, 12, 13, 14, 15]

In [41]:
### Hoặc sử dụng statisticmethod vì statisticmethod ko thuộc về đổi tượng nào 

class MyClass_1:
  def __init__(self,value):
    self.value = value 
  @staticmethod
  def add(x):
    return x+10 
result = rdd.map(MyClass_1.add)
print(result.collect())

[11, 12, 13, 14, 15]


----------------------------------------------------
<b>Common Transformations and Actions</b> 

<b>Transformation</b>

In [42]:
## map()
rdd = sc.parallelize([1,2,3,4,5,5])
rdd.map(lambda x: x*2).collect()


[2, 4, 6, 8, 10, 10]

In [44]:
rdd = sc.parallelize(["Xin chao Ha Noi"])
rdd.map(lambda x: x.split(" ")).collect()


[['Xin', 'chao', 'Ha', 'Noi']]

In [54]:
## flatMap()
rdd = sc.parallelize([1,2])
rdd.flatMap(lambda x: [x+5]).collect()


[6, 7]

In [57]:
rdd = sc.parallelize(["VN thật đẹp","VN quá đẹp"])
rdd.flatMap(lambda x: x.split(" ")).collect()

['VN', 'thật', 'đẹp', 'VN', 'quá', 'đẹp']

In [58]:
## filter () :  lọc theo điều  kiện trong ()

In [59]:
## distinct(): xóa phần tử trùng 

In [60]:
## Transformation 2 RDD -> 1 RDD
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([3,4,5])  

In [61]:
rdd1.union(rdd2).collect()

[1, 2, 3, 3, 4, 5]

In [62]:
rdd1.intersection(rdd2).collect()

[3]

In [63]:
rdd1.subtract(rdd2).collect()

[2, 1]

In [64]:
rdd1.cartesian(rdd2).collect()

[(1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5)]

<b> Action</b>

In [67]:
rdd = sc.parallelize([1,2,3,4])

In [68]:
rdd.collect()

[1, 2, 3, 4]

In [70]:
rdd.count()

4

In [71]:
rdd.countByValue()

defaultdict(int, {1: 1, 2: 1, 3: 1, 4: 1})

In [74]:
rdd.take(3)

[1, 2, 3]

In [75]:
rdd.top(3)

[4, 3, 2]

----------------------------------------------------
<b>Note</b> 
Một số hàm chỉ dùng cho 1 số loại RDD nhất định

Ví dụ:
* mean(),variance() chỉ dùng cho RDD số 
* join() chỉ dùng cho key/value RDD

----------------------------------------------------

<b>Note</b> 

Do các <b>transformation</b> của RDD là lười nền cần lưu ý khi thực hiện <b>actions</b> có thể khiến RDD thực hiện lại transformation nhiều gần gây lãng phí tài nguyên  
